# Let's Chat(bot)

This is where I'm going to try to build a chatbot. Not from scratch, mind you, I'll be using the Chatterbot Python library. Let's install it, and see if we'll be good to go.

In [1]:
#pip install chatterbot

Looks good so far, now importing the library

In [1]:
from chatterbot import ChatBot
import spacy

In [2]:
spacy.prefer_gpu()

False

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
bot=ChatBot('Bender')

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")